## Load data from csv to pandas dataframe

In [2]:
import pandas as pd

# Load data_categories as dataframe
data_categories = pd.read_csv('.\\data\\raw\\data_categories.csv', index_col=False, delimiter = ';')

# Load data_market_prices as dataframe
data_market_prices = pd.read_csv('.\\data\\raw\\data_market_prices.csv', index_col=False, low_memory=False, delimiter = ',')

# Load data_vas_purchases as dataframe
data_vas_purchases = pd.read_csv('.\\data\\raw\\data_vas_purchases.csv', index_col=False, delimiter = ';')

## Connect to MySQL server, create new database and necessary tables

In [52]:
import pandas as pd
from sqlalchemy import create_engine

# Credentials to database connection
hostname="localhost"
dbname="mydatabase"
uname="tomek"
pwd="tomek123"

# Connect to server
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}"
				.format(user=uname, pw=pwd, host=hostname))

# Remove database if exist
engine.execute("DROP DATABASE IF EXISTS {db}"
                .format(db=dbname))

# Create database
engine.execute("CREATE DATABASE {db}"
                .format(db=dbname))

# Select new database
engine.execute("USE {db}"
                .format(db=dbname))

# Drop tables if exist
engine.execute("""
					DROP TABLE IF EXISTS data_categories,data_market_prices,data_vas_purchases;
				""")

# Convert dataframes to sql tables                                 
data_categories.to_sql('data_categories', engine, index=False)
data_market_prices.to_sql('data_market_prices', engine, index=False)
data_vas_purchases.to_sql('data_vas_purchases', engine, index=False)

# Rename original columns to save original values and create new columns with date-type data
queries = ("ALTER TABLE data_market_prices RENAME COLUMN date_posted TO date_posted_old;",              # rename date_posted column to date_posted_old
           "ALTER TABLE data_market_prices RENAME COLUMN date_expired TO date_expired_old;",            # rename date_expired column to date_expired_old
           "ALTER TABLE data_market_prices ADD date_posted DATE AFTER date_posted_old;",                # create new date_posted column to store date-type data
           "ALTER TABLE data_market_prices ADD date_expired DATE AFTER date_expired_old;",              # create new date_expired column to store date-type data
           "UPDATE data_market_prices SET date_posted = STR_TO_DATE(date_posted_old,'%%c/%%e/%%Y');",   # fill new column with data 
           "UPDATE data_market_prices SET date_expired = STR_TO_DATE(date_expired_old,'%%c/%%e/%%Y');"  # fill new column with data
            )

for query in queries:
    engine.execute(query)

# 3/28/2022	
# engine.execute("""
#                 ALTER TABLE data_market_prices RENAME COLUMN date_posted TO date_posted_old;
#                 """)
#"UPDATE data_market_prices SET date_expired=TO_DATE(date_expired_old,'MM/DD/YYYY');"


                #ALTER TABLE data_market_prices RENAME COLUMN date_expired TO date_expired_old;
                #ALTER TABLE my_table ADD (new_col DATE);
                #UPDATE my_table SET new_col=TO_DATE(old_col,'MM/DD/YYYY');
                #ALTER TABLE data_market_prices ADD date_posted DATE;

                # UPDATE data_market_prices SET date_posted=TO_DATE(date_posted_old,'MM/DD/YYYY');

                # ALTER TABLE data_market_prices ADD (date_expired DATE);
                # UPDATE data_market_prices SET date_expired=TO_DATE(date_expired_old,'MM/DD/YYYY');

## Analyse data

In [58]:
import pandas as pd
from sqlalchemy import create_engine

# Credentials to database connection
hostname="localhost"
dbname="mydatabase"
uname="tomek"
pwd="tomek123"

# Connect to server
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
				.format(user=uname, pw=pwd, host=hostname, db=dbname))

clean_data_market_price = pd.read_sql("""
										SELECT *
										FROM data_market_prices
										WHERE ad_id IS NOT NULL
											AND city_id IS NOT NULL
											AND category_id IS NOT NULL
											AND market IS NOT NULL
											AND date_posted_old IS NOT NULL
											AND date_expired_old IS NOT NULL
											AND price IS NOT NULL
											AND rooms_num IS NOT NULL
											AND price_per_sqm IS NOT NULL
											AND DATEDIFF(date_expired, date_posted) >= 0;
									""", engine)

#pd.set_option('display.expand_frame_repr', False) convert(date_expired, getdate(), 1)

In [56]:
clean_data_market_price.head()

,ad_id,city_id,category_id,market,date_posted_old,date_posted,date_expired_old,date_expired,price_per_sqm,price,rooms_num
0,62925670,26,101,primary,3/28/2022,2022-03-28,5/12/2022,2022-05-12,15968.0,1390000.0,3
1,62261719,1004,101,secondary,9/9/2021,2021-09-09,3/4/2022,2022-03-04,6743.0,499000.0,3
2,61969443,26,101,secondary,6/17/2021,2021-06-17,6/9/2022,2022-06-09,14007.0,997000.0,3
3,62569893,26,101,secondary,12/9/2021,2021-12-09,2/15/2022,2022-02-15,31238.0,2499000.0,3
4,62391825,26,101,primary,10/15/2021,2021-10-15,6/12/2022,2022-06-12,9790.0,600000.0,3


In [59]:
clean_data_market_price.shape

(77298, 11)

In [27]:
clean_data_market_price.shape
clean_data_market_price.head()

,ad_id,city_id,category_id,market,date_posted,date_expired,price_per_sqm,price,rooms_num
0,62261719,1004,101,secondary,9/9/2021,3/4/2022,6743.0,499000.0,3
1,61790505,1004,102,secondary,4/28/2021,3/24/2022,1531.0,49000.0,1
2,61960498,26,101,secondary,6/15/2021,2/1/2022,11600.0,435000.0,2
3,62215417,1004,101,primary,8/26/2021,2/10/2022,7784.0,413900.0,3
4,62099315,26,101,secondary,7/23/2021,6/24/2022,9235.0,1099000.0,5


In [19]:
raw = pd.read_sql("""
					SELECT *
					FROM data_market_prices;
				""", engine)


In [20]:
raw.shape

(105062, 9)